<a href="https://colab.research.google.com/github/Surajpatra700/Deep-Learning_Project/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ******************************************************************* INPUT PIPELINES *****************************************************

In [2]:
# In deep learning, an input pipeline is a set of processes that efficiently loads, preprocesses, and feeds data (e.g., images, text, or other types)
# into a neural network for training or inference. It ensures data is ready for the model, handles data augmentation, batching, and other tasks
# to optimize the training process.

In [3]:
import tensorflow as tf

In [4]:
daily_sales_number = [21,22,-108,31,-1,32,34,31]

In [5]:
#  Creating my dataset
tf_dataset= tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [6]:
for sales in tf_dataset:
  print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [7]:
# taking only 3 Number from the List

for sales in tf_dataset.take(3):
  print(sales.numpy())

21
22
-108


In [8]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset:
  print(sales.numpy())

21
22
31
32
34
31


In [9]:
# USD to India Rupees conversion

tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset:
  print(sales.numpy())

1512
1584
2232
2304
2448
2232


In [10]:
tf_datset = tf_dataset.shuffle(3)
for sales in tf_dataset:
  print(sales.numpy())

1512
1584
2232
2304
2448
2232


In [11]:
for sales_batch in tf_dataset.batch(3):
  print(sales_batch.numpy())

[1512 1584 2232]
[2304 2448 2232]


In [12]:
#  Creating all these set of processes in one single line

In [13]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(3)

for sales in tf_dataset:
  print(sales.numpy())

[1584 1512 2304]
[2232 2448 2232]


In [14]:
# **************************************** TENSORFLOW DATASETS ******************************

In [15]:
import numpy
import tensorflow as tf

In [16]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
(X_train.shape, y_train.shape) , (X_test.shape, y_test.shape)

11490434/11490434 [==============================] - 1s 0us/step


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [17]:
example_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
example_dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [18]:
for (img, label) in example_dataset:
  print(img.numpy().shape, label.numpy())
  break

(28, 28) 5


In [19]:
def normalize_img(image, label):
  return (tf.cast(image, tf.float32)/255.0 , label)

example_dataset = example_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

example_dataset

<_ParallelMapDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [20]:
example_dataset = example_dataset.shuffle(len(example_dataset))

for (img, label) in example_dataset:
  print(img.numpy().shape, label.numpy())
  break

(28, 28) 5


In [21]:
# Batching the dataset

In [22]:
example_dataset = example_dataset.batch(60)
example_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [23]:
for (img, label) in example_dataset:
  print(img.numpy().shape, label.numpy())
  break

(60, 28, 28) [0 3 2 1 0 6 7 7 7 2 2 4 3 0 9 9 4 7 9 0 6 0 3 4 9 1 7 9 5 1 3 9 2 0 4 3 9
 9 3 8 2 1 2 4 1 6 3 6 2 6 9 1 1 2 8 6 8 4 8 2]


In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

train_dataset = train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.cache() # cache() helps to store the dataset in cache i.e small high speed memory

train_dataset = train_dataset.shuffle(len(train_dataset))
train_dataset = train_dataset.batch(50)

train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [25]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

test_dataset = test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(50)
test_dataset = test_dataset.cache() # cache() helps to store the dataset in cache i.e small high speed memory

test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.InputLayer(input_shape=(28,28)))
model.add(layers.Reshape((28,28,1)))
model.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 8)         584       
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 8)         584       
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 8)         584       
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 8)         584       
                                                                 
 global_average_pooling2d (  (None, 8)                 0

In [27]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5) # If validation loss doesnot improve after 5 epochs it will giveUp.

In [29]:
model.fit(
    train_dataset,
    epochs=10,
    validation_data = test_dataset,
    callbacks=[es]
)

Epoch 1/10
1200/1200 [==============================] - 23s 6ms/step - loss: 1.7533 - accuracy: 0.3323 - val_loss: 1.2299 - val_accuracy: 0.5511
Epoch 2/10
1200/1200 [==============================] - 8s 7ms/step - loss: 1.1019 - accuracy: 0.5852 - val_loss: 0.9776 - val_accuracy: 0.6220
Epoch 3/10
1200/1200 [==============================] - 7s 6ms/step - loss: 0.8177 - accuracy: 0.7024 - val_loss: 0.6629 - val_accuracy: 0.7702
Epoch 4/10
1200/1200 [==============================] - 8s 6ms/step - loss: 0.5908 - accuracy: 0.7976 - val_loss: 0.4959 - val_accuracy: 0.8298
Epoch 5/10
1200/1200 [==============================] - 8s 7ms/step - loss: 0.4540 - accuracy: 0.8513 - val_loss: 0.5389 - val_accuracy: 0.8132
Epoch 6/10
1200/1200 [==============================] - 7s 6ms/step - loss: 0.3837 - accuracy: 0.8764 - val_loss: 0.3528 - val_accuracy: 0.8860
Epoch 7/10
1200/1200 [==============================] - 8s 7ms/step - loss: 0.3466 - accuracy: 0.8894 - val_loss: 0.3134 - val_accuracy